In [29]:
import time
import pymysql
from sqlalchemy import create_engine
import sqlalchemy
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

In [30]:
# caps = DesiredCapabilities().CHROME
# caps["pageLoadStrategy"] = "none"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [60]:
def cody_scraping():
    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.implicitly_wait(3)
    codyDf = pd.DataFrame(columns={"cody_category"})
    clothesDf = pd.DataFrame(columns=("cody_id", "clothes_maincategory", "clothes_subcategorry", "brand", "color", "season", "gender",
                                        "spring", "summer", "autumn", "winter", "img"))
    reviewDf = pd.DataFrame(columns=("clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"))

    codyCategory = ["americancasual"] # , "casual", "chic", "dandy", "formal", "girlish", "golf", "retro", "romantic", "sports", "street"]
    codyIdx = 0;
    clothesIdx = 0;
    for category in codyCategory:
        for i in range(1, 2): #3
            url = "https://www.musinsa.com/app/styles/lists?use_yn_360=&style_type=%s&brand=&model=&tag_no=&max_rt=&min_rt=&display_cnt=60&list_kind=big&sort=view_cnt&page=%d" % (category, i)
            for j in range(1, 2): #61
                wd.get(url)
                wd.find_element(By.XPATH, "/html/body/div[3]/div[3]/form/div[4]/div/ul/li[%d]/div[1]/a" % j).click()
                codyDf.loc[codyIdx] = [category]
                codyIdx += 1
                ###################################################
                # 옷 정보 크롤링
                ###################################################
                for info in range(1, 11):
                    print(info)
                    info_url = ""
                    try:
                        info_url = wd.find_element(By.XPATH, '//*[@id="style_info"]/div[3]/div[2]/div/div/div[1]/div[%d]/div[1]/a'%info).get_attribute('href')
                    except:
                        print(check)
                        continue

                    wd.get(info_url)

                    clothesMaincategory = wd.find_element(By.XPATH,'//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[1]').text
                    clothesSubCategory = wd.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[2]').text
                    check = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/h4').text
                    img = wd.find_element(By.XPATH, '//*[@id="bigimg"]').get_attribute('src')

                    if(check == "Product Info제품정보"):
                        brand = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[1]/p[2]/strong/a').text
                        try:
                            season = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/strong').text
                        except:
                            season = 0
                        if(season == 0):
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span').text
                        else:
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span[2]').text
                    else:
                        brand = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[1]/p[2]/strong/a').text
                        try:
                            season = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/strong').text
                        except:
                            season = 0

                        if(season == 0):
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/span').text
                        else:
                            gender = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/span[2]').text
                    # clothes_maincategory clothes_subcategorry  summer autumn spring  gender brand cody_id color season winter
                    clothesDf.loc[clothesIdx] = [(codyIdx-1), clothesMaincategory, clothesSubCategory, brand, np.nan, season, gender, np.nan, np.nan, np.nan, np.nan, img]

                    tempDf = review_crawling(clothesIdx, wd)
                    reviewDf = pd.concat([reviewDf, tempDf], ignore_index=True)
                    print(reviewDf)
                    clothesIdx += 1

                    wd.back()
            # print(codyDf.tail())
    return codyDf, clothesDf, reviewDf

In [66]:
def review_crawling(clothesIdx, wd):
    reviewDf = pd.DataFrame(columns=("clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"))
    wd.find_element(By.XPATH, '//*[@id="estimateBox"]/div[2]/ul/li[3]').click()

    reviewIdx = 0
    last = False
    for i in range(1, 3):
        if(last):
            break
        for page in range(1, 6):
            if(last):
                break
            for idx in range(1, 12):
                if(last):
                    break

                try:
                    text = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[1]/p[1]' % idx).text
                except:
                    if(idx <= 10):
                        last = True
                    else:
                        if(page == 5):
                            break
                        wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' %  (idx, (page + 3))).send_keys(Keys.ENTER)

                try:
                    user = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[2]/p' % idx).text
                except:
                    continue
                userArr = user.split(',')
                gender = userArr[0]
                height = userArr[1]
                weight = userArr[2]
                size = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div[2]/p/span' % idx).text
                size_review = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[1]/span'% idx).text
                bright = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[2]/span'% idx).text
                color = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[3]/span'% idx).text
                thick = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[4]/span'% idx).text

                reviewDf.loc[reviewIdx] = [clothesIdx, gender, height, weight, size, size_review, bright, color, thick]
                reviewIdx += 1


        if(last == False):
            wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[8]').click()
    return reviewDf



In [67]:
cody, clothes, review = cody_scraping()

1


ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00DEDF13+2219795]
	Ordinal0 [0x00D82841+1779777]
	Ordinal0 [0x00C94100+803072]
	Ordinal0 [0x00CC3FB6+999350]
	Ordinal0 [0x00CB9B76+957302]
	Ordinal0 [0x00CDE7FC+1107964]
	Ordinal0 [0x00CB94B4+955572]
	Ordinal0 [0x00CDEA14+1108500]
	Ordinal0 [0x00CEF192+1175954]
	Ordinal0 [0x00CDE616+1107478]
	Ordinal0 [0x00CB7F89+950153]
	Ordinal0 [0x00CB8F56+954198]
	GetHandleVerifier [0x010E2CB2+3040210]
	GetHandleVerifier [0x010D2BB4+2974420]
	GetHandleVerifier [0x00E86A0A+565546]
	GetHandleVerifier [0x00E85680+560544]
	Ordinal0 [0x00D89A5C+1808988]
	Ordinal0 [0x00D8E3A8+1827752]
	Ordinal0 [0x00D8E495+1827989]
	Ordinal0 [0x00D980A4+1867940]
	BaseThreadInitThunk [0x7520FA29+25]
	RtlGetAppContainerNamedObjectPath [0x76F27A9E+286]
	RtlGetAppContainerNamedObjectPath [0x76F27A6E+238]


In [58]:
review

,clothes_id,gender,height,weight,size,size_review,bright,color,thick


In [164]:
############################################################
#리뷰 크롤링
############################################################
def review_crawling2(reviewDf, clothesIdx, wd):
    last = False
    reviewIdx = 0
    for jump in range(1, 3):
        print(jump)
        for page in range(1,6):
            lastIdx = 1
            print("page", page)
            for idx in range(1, 11):
                print('idx', idx)
                lastIdx = idx
                try:
                    text = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[1]/p[1]'% idx).text
                    print(text)
                except:
                    print("여기에요")
                    if(idx < 10):
                        last = True
                    break

                # 성별, 키, 몸무게
                try:
                    user = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[1]/div/div[2]/p' % idx).text
                except:
                    continue

                user = user.split(",")
                print('user', user)
                reGender = user[0]
                height = user[1]
                weight = user[2]
                size = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div[2]/p/span' % idx).text
                sizeReview = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[1]/span' % idx).text
                bright = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[2]/span' % idx).text
                color = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[3]/span' % idx).text
                thick = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[4]/div[2]/ul/li[4]/span' % idx).text

                #"clothes_id", "gender", "height", "weight", "size", "size_review", "bright", "color", "thick"
                reviewDf.loc[reviewIdx] = [clothesIdx, reGender, height, weight, size, sizeReview, bright, color, thick]
                reviewIdx += 1
            print(last)
            if(last) :
                break;
            if(lastIdx < 10):
                print('lastInx', 1, lastIdx)
                pageNum = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' % ((lastIdx), (page + 3))).text
            else:
                print('lastInx', 2, lastIdx)
                pageNum = wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' % ((lastIdx+1), (page + 3))).text

            if(pageNum.isdigit()):
                wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[%d]/div[2]/div/a[%d]' % ((lastIdx+1), (page + 3))).send_keys(Keys.ENTER)
            elif (int(pageNum.isdigit()) - 1 < 8 ):
                wd.find_element(By.XPATH, '//*[@id="reviewListFragment"]/div[11]/div[2]/div/a[6]').send_keys(Keys.ENTER)
        print(last)
        if(last) :
            break;



    return reviewDf

In [45]:
url = 'https://www.musinsa.com/app/goods/2232639/0'
wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(url)
text = wd.find_element(By.XPATH, '//*[@id="product_order_info"]/div[2]/ul/li[2]/p[2]/span').text
print(text)

남 여 라이프
